In [1]:
# Importing the libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Generating final dataset for Pueblo, CO

In [2]:
# reading air temperature and precipitation data
df_air_temp_precip = pd.read_parquet('data/airtemp_precip_data/pueblo.parquet')

# reading qwl data
df_gwl = pd.read_parquet('data/gwl_data/pueblo.parquet')

# reading streamflow data
df_streamflow = pd.read_parquet('data/streamflow_data/fountain_creek.parquet')

# reading soil moisture and temperature data
df_soil = pd.read_parquet('data/soiltemp_soilmoisture_data/soil_temp_moisture.parquet')

# converting date object to datetime
df_air_temp_precip['date'] = pd.to_datetime(df_air_temp_precip['date'])
df_gwl['date'] = pd.to_datetime(df_gwl['date'])
df_streamflow['date'] = pd.to_datetime(df_streamflow['date'])
df_soil['date'] = pd.to_datetime(df_soil['date'])

In [3]:
# checking for min and max dates for each dataset

print('Air temperature and precipitation data')
print('Min date:', df_air_temp_precip.date.min())
print('Max date:', df_air_temp_precip.date.max())

print('GWL data')
print('Min date:', df_gwl.date.min())
print('Max date:', df_gwl.date.max())

print('Streamflow data')
print('Min date:', df_streamflow.date.min())
print('Max date:', df_streamflow.date.max())

print('Soil moisture and temperature data')
print('Min date:', df_soil.date.min())
print('Max date:', df_soil.date.max())


Air temperature and precipitation data
Min date: 2009-01-01 00:00:00
Max date: 2024-02-08 00:00:00
GWL data
Min date: 2009-01-01 00:00:00
Max date: 2024-02-08 00:00:00
Streamflow data
Min date: 2009-01-01 00:00:00
Max date: 2024-02-08 00:00:00
Soil moisture and temperature data
Min date: 2009-11-21 00:00:00
Max date: 2023-10-03 00:00:00


In [4]:
# merging all datasets

df_pueblo = df_air_temp_precip.merge(df_gwl, on='date', how='inner')
df_pueblo = df_pueblo.merge(df_soil, on='date', how='inner')
df_pueblo = df_pueblo.merge(df_streamflow, on='date', how='inner')



# filtering df to be from 2010
df_pueblo = df_pueblo[df_pueblo['date'].dt.year >= 2010]

# dropping rows with missing values
df_pueblo = df_pueblo.dropna()

df_pueblo.head()

,date,precip,max_temp,min_temp,gwl,sm_20cm,st_20cm,sm_50cm,st_50cm,sm_100cm,st_100cm,daily_discharge_cfs
85,2010-02-14,0.0,36,13,20.52,0.205500,0.408333,0.182875,1.320833,0.135333,3.700000,136.0
86,2010-02-15,0.0,41,7,20.51,0.215333,0.595833,0.183292,1.445833,0.137250,3.625000,129.0
87,2010-02-16,0.0,43,8,20.51,0.210417,0.500000,0.183625,1.495833,0.136583,3.700000,130.0
88,2010-02-17,0.0,51,10,20.51,0.209708,0.500000,0.184667,1.437500,0.135625,3.645833,132.0
89,2010-02-18,0.0,51,16,20.50,0.210542,0.637500,0.185125,1.479167,0.135333,3.708333,135.0


### Drought Data from NIDIS (National Integrated Drought Information System)

In [12]:
# Drought data for Colorado
# Source: https://drought.gov/states/colorado#historical-conditions
# Data Descroiption: https://www.drought.gov/data-maps-tools/historical-drought-data-conditions-tool

df = pd.read_csv('data/drought_data/data.csv')

# deletiong StateAbbreviation, StatisticFormatID, MapDate columns
df = df.drop(['StateAbbreviation', 'StatisticFormatID', 'MapDate'], axis=1)

# creating a new column date, for all dates between ValidStart and ValidEnd and replicating the row for each date

df['ValidStart'] = pd.to_datetime(df['ValidStart'])
df['ValidEnd'] = pd.to_datetime(df['ValidEnd'])

df['date'] = df.apply(lambda x: pd.date_range(start=x['ValidStart'], end=x['ValidEnd'], freq='D'), axis=1)
df = df.explode('date')
df = df.drop(['ValidStart', 'ValidEnd'], axis=1)

# Sorting the dataframe by date
df = df.sort_values(by='date')

# Dropping rows with duplicate dates
df = df.drop_duplicates(subset='date', keep='first')

# Resetting the index
df = df.reset_index(drop=True)

df.head()

,None,D0,D1,D2,D3,D4,date
0,96.81,3.19,0.0,0.0,0.0,0.0,2000-01-04
1,96.81,3.19,0.0,0.0,0.0,0.0,2000-01-05
2,96.81,3.19,0.0,0.0,0.0,0.0,2000-01-06
3,96.81,3.19,0.0,0.0,0.0,0.0,2000-01-07
4,96.81,3.19,0.0,0.0,0.0,0.0,2000-01-08


In [13]:
# Saving the merged dataframe as a parquet file

df_pueblo.to_parquet('data/final_data/pueblo.parquet')
df.to_parquet('data/final_data/target.parquet')